In [22]:
'''
4.4.3 VRP con split-delivery (C=30 kg), vuelos directos (>8 km) y OR-Tools

Este script resuelve un problema de ruteo de vehículos (VRP) adaptado a drones:
- Capacidad máxima de cada dron (C)
- Separación de paradas lejanas (> R_max) como vuelos directos
- Modelado split-delivery para servir demandas fragmentadas
- Uso de OR-Tools para optimizar rutas de nodos cercanos
- Cálculo posterior de tiempos de vuelo y carga total servida
- Permite elegir el día de simulación mediante un índice
'''
import os
import pandas as pd
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# ------------------------------------------------
# 1) Parámetros del modelo
# ------------------------------------------------
C      = 30           # capacidad de carga por dron [kg]
R_max  = 8000         # umbral para vuelos directos [m]
nd     = 3            # número de drones disponibles
turns  = 8            # salidas por dron
nv     = nd * turns   # vuelos virtuales totales (24)
i_dia  = 0            # índice del día a simular

# ------------------------------------------------
# 2) Carga de datos y selección de día
# ------------------------------------------------
dem_split = pd.read_csv('../data/simulated_demands_split.csv')
fechas    = dem_split['date'].unique()
fecha_sel = fechas[i_dia]
day_df    = dem_split[dem_split['date'] == fecha_sel].reset_index(drop=True)
print(f"Simulando para la fecha: {fecha_sel}")

# ------------------------------------------------
# 3) Carga de coordenadas y nombres de nodos
# ------------------------------------------------
paradas_df  = pd.read_csv('../data/PARADAS_CON_HABITANTES.csv')
coords_dep  = np.array([[570886.225, 4682838.781]])  # depósito
stop_coords = {r['Name']: (r['stop_x'], r['stop_y']) for _, r in paradas_df.iterrows()}

names  = ['DEPOT_RIBADAVIA']
coords = [coords_dep[0]]
for s in day_df['stop']:
    names.append(s)
    coords.append(stop_coords[s])
coords = np.vstack(coords)

# ------------------------------------------------
# 4) Matriz local de distancias (euclídea)
# ------------------------------------------------
N        = len(names)
dist_mat = np.linalg.norm(coords[:, None, :] - coords[None, :, :], axis=2)

# ------------------------------------------------
# 5) Demandas por nodo
# ------------------------------------------------
demand_list = [0] + day_df['simulated_demand_kg'].astype(int).tolist()

# ------------------------------------------------
# 6) Vuelos directos (> R_max)
# ------------------------------------------------
far_idx       = [i for i in range(1, N) if dist_mat[0, i] > R_max]
routes_direct = []
for j in far_idx:
    d = dist_mat[0, j]
    routes_direct.append({
        'vehicle_orig': None,  # placeholder; reasignaremos luego
        'route':        f"DEPOT → {names[j]} → DEPOT",
        'distance_m':   int(2 * d),
        'distance_km':  round((2 * d) / 1000, 2),
        'demand_kg':    demand_list[j]
    })
n_directs = len(routes_direct)
for idx, rec in enumerate(routes_direct, start=1):
    rec['vehicle_orig'] = idx

# ------------------------------------------------
# 7) Subproblema para nodos cercanos (≤ R_max)
# ------------------------------------------------
near_idx = [i for i in range(N) if i not in far_idx]
map_idx  = {orig: i for i, orig in enumerate(near_idx)}
inv_map  = {i: orig for orig, i in map_idx.items()}

sub_dist  = dist_mat[np.ix_(near_idx, near_idx)].astype(int)
sub_dem   = [demand_list[i] for i in near_idx]
sub_names = [names[i] for i in near_idx]

nv_sub    = nv - n_directs

# ------------------------------------------------
# 8) Configuración OR-Tools para nodos cercanos
# ------------------------------------------------
mgr = pywrapcp.RoutingIndexManager(len(sub_dist), nv_sub, map_idx[0])
rt  = pywrapcp.RoutingModel(mgr)

def cb_dist(frm, to):
    return sub_dist[mgr.IndexToNode(frm), mgr.IndexToNode(to)]
idx_dist = rt.RegisterTransitCallback(cb_dist)
rt.SetArcCostEvaluatorOfAllVehicles(idx_dist)

def cb_dem(frm):
    return sub_dem[mgr.IndexToNode(frm)]
idx_dem = rt.RegisterUnaryTransitCallback(cb_dem)
rt.AddDimensionWithVehicleCapacity(idx_dem, 0, [C] * nv_sub, True, 'Capacity')

params = pywrapcp.DefaultRoutingSearchParameters()
params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
params.time_limit.seconds = 20

sol = rt.SolveWithParameters(params)
if not sol:
    raise RuntimeError("Sin solución factible para nodos cercanos.")

# ------------------------------------------------
# 9) Extracción de rutas: directas + cercanas
# ------------------------------------------------
routes = [r.copy() for r in routes_direct]

for v in range(nv_sub):
    start_idx = rt.Start(v)
    idx       = sol.Value(rt.NextVar(start_idx))
    prev, dist_v, weight_v = map_idx[0], 0, 0
    seq = ['DEPOT_RIBADAVIA']
    while not rt.IsEnd(idx):
        node = mgr.IndexToNode(idx)
        orig = inv_map[node]
        if orig != 0:
            seq.append(sub_names[node])
            weight_v += demand_list[orig]
        dist_v += sub_dist[prev, node]
        prev, idx = node, sol.Value(rt.NextVar(idx))
    dist_v += sub_dist[prev, map_idx[0]]  # retorno al depósito
    seq.append('DEPOT_RIBADAVIA')
    routes.append({
        'vehicle_orig': n_directs + v + 1,
        'route':        " → ".join(seq),
        'distance_m':   int(dist_v),
        'distance_km':  round(dist_v / 1000, 2),
        'demand_kg':    weight_v
    })

# ------------------------------------------------
# 10) Post-procesado y reordenación de rutas
# ------------------------------------------------
df = pd.DataFrame(routes)

# 10.1 Calcular total_time_min; rutas sin vuelo = 0
df['total_time_min'] = df['distance_m'] / 15 / 60 + 2
df.loc[df['distance_m'] == 0, 'total_time_min'] = 0

# 10.2 Redondeos
df['distance_km']    = df['distance_km'].round(2)
df['total_time_min'] = df['total_time_min'].round(2)

# 10.3 Etiquetar tipo de ruta según distance_m y vehicle_orig
def etiquetar_tipo(row):
    if row['vehicle_orig'] <= n_directs:
        return "Directa"
    elif row['distance_m'] > 0:
        return "Concatenada"
    else:
        return "Sin vuelo"
df['tipo_ruta'] = df.apply(etiquetar_tipo, axis=1)

# 10.4 Asignar prioridad numérica: Directa=1, Concatenada=2, Sin vuelo=3
prioridad = {"Directa": 1, "Concatenada": 2, "Sin vuelo": 3}
df['orden'] = df['tipo_ruta'].map(prioridad)

# 10.5 Ordenar por 'orden' (prioridad) y luego por 'vehicle_orig'
df = df.sort_values(by=['orden', 'vehicle_orig'], ascending=[True, True]).reset_index(drop=True)

# 10.6 Reasignar vehicle en orden secuencial a partir de 1
df_activo   = df[df['distance_m'] > 0].copy()
df_no_vuelo = df[df['distance_m'] == 0].copy()

df_activo['vehicle'] = np.arange(1, len(df_activo) + 1)
df_no_vuelo['vehicle'] = np.arange(len(df_activo) + 1, len(df_activo) + len(df_no_vuelo) + 1)

# 10.7 Reemplazar rutas sin vuelo con texto
df_no_vuelo['route'] = "VUELO SIN RUTA ASIGNADA"

# 10.8 Concatenar activos y “sin vuelo” en orden
df_concat = pd.concat([df_activo, df_no_vuelo], ignore_index=True)

# 10.9 Selección de columnas finales
df_clean = df_concat[[
    'vehicle',
    'route',
    'distance_km',
    'distance_m',
    'demand_kg',
    'total_time_min'
]].copy()

# 10.10 Agregar fila de TOTAL al final
total_dist_km  = df_clean['distance_km'].sum()
total_dist_m   = df_clean['distance_m'].sum()
total_load_kg  = df_clean['demand_kg'].sum()
total_time_min = df_clean['total_time_min'].sum()

total_row = {
    'vehicle':        'TOTAL',
    'route':          '',
    'distance_km':    round(total_dist_km, 2),
    'distance_m':     int(total_dist_m),
    'demand_kg':      int(total_load_kg),
    'total_time_min': round(total_time_min, 2)
}

df_final = pd.concat([df_clean, pd.DataFrame([total_row])], ignore_index=True)

# ------------------------------------------------
# 11) Guardar resultado en CSV
# ------------------------------------------------
os.makedirs('../results', exist_ok=True)
df_final.to_csv('../results/routes_mixed.csv', index=False)

print(f"✅ Rutas para {fecha_sel} guardadas en ../results/routes_mixed.csv")
print("Resumen TOTAL incluido en la última fila de la tabla.")
print(f"Tiempo total de vuelo: {total_time_min:.2f} min")

Simulando para la fecha: 2025-05-31
✅ Rutas para 2025-05-31 guardadas en ../results/routes_mixed.csv
Resumen TOTAL incluido en la última fila de la tabla.
Tiempo total de vuelo: 337.22 min
